In [20]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [21]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [22]:
soup_corona = bs(html_corona, 'html.parser')

In [23]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [24]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewCases"].astype("float")
corona_df["Date"]= dt.datetime.today()

In [25]:
corona_df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,Reported1st case,Date
0,USA,142735,275.0,2489.0,5.0,4562.0,135684,2970.0,431.0,8.0,Jan 20,2020-03-30 05:03:32.157685
1,Italy,97689,0.0,10779.0,0.0,13030.0,73880,3906.0,1616.0,178.0,Jan 29,2020-03-30 05:03:32.157685
2,Spain,80110,0.0,6803.0,0.0,14709.0,58598,4165.0,1713.0,146.0,Jan 30,2020-03-30 05:03:32.157685
3,Germany,62435,0.0,541.0,0.0,9211.0,52683,1979.0,745.0,6.0,Jan 26,2020-03-30 05:03:32.157685
4,France,40174,0.0,2606.0,0.0,7202.0,30366,4632.0,615.0,40.0,Jan 23,2020-03-30 05:03:32.157685
...,...,...,...,...,...,...,...,...,...,...,...,...
196,Saint Kitts and Nevis,2,0.0,0.0,0.0,0.0,2,0.0,38.0,0.0,Mar 24,2020-03-30 05:03:32.157685
197,Papua New Guinea,1,0.0,0.0,0.0,0.0,1,0.0,0.1,0.0,Mar 19,2020-03-30 05:03:32.157685
198,St. Vincent Grenadines,1,0.0,0.0,0.0,1.0,0,0.0,9.0,0.0,Mar 10,2020-03-30 05:03:32.157685
199,Timor-Leste,1,0.0,0.0,0.0,0.0,1,0.0,0.8,0.0,Mar 20,2020-03-30 05:03:32.157685


In [26]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)
#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)
df_used["ClosedCases"] = df_used["TotalDeaths"] +  df_used["TotalRecovered"]

df_used["ClosedDeathRate"] = df_used["TotalDeaths"]/df_used["ClosedCases"]
df_used["ClosedDeathRate"].fillna(0,inplace = True)

df_used["CurrentDeathRate"] = df_used["TotalDeaths"]/df_used["TotalCases"]
df_used["CurrentDeathRate"].fillna(0,inplace = True)

total_cases = df_used["TotalCases"].sum()
total_closed_cases = df_used["ClosedCases"].sum()

df_used["Weight"] = [x/total_closed_cases for x in df_used["ClosedCases"]]

# #checking if total weight is 1 
# validate_weight = df_used["Weight"].sum()


df_used["DeathRateXWeight"] = df_used["CurrentDeathRate"] *df_used["Weight"]
df_used["DeathRateXWeight"].sum()
df_used["notes"] = ["Last 8 columns are added by Roaya" for x in df_used["DeathRateXWeight"]  ]
df_used.sort_values(by = "DeathRateXWeight", ascending = False)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Deaths/1M pop,Reported1st case,Date,Longitude,Latitude,ClosedCases,ClosedDeathRate,CurrentDeathRate,Weight,DeathRateXWeight,notes
172,China,81470,31.0,3304.0,4.0,75700.0,2466,633.0,57.0,2.0,Jan 10,2020-03-30 05:03:32.157685,103.819074,36.561765,79004.0,0.041821,0.040555,0.426256,0.017287,Last 8 columns are added by Roaya
1,Italy,97689,0.0,10779.0,0.0,13030.0,73880,3906.0,1616.0,178.0,Jan 29,2020-03-30 05:03:32.157685,12.070013,42.796626,23809.0,0.452728,0.110340,0.128458,0.014174,Last 8 columns are added by Roaya
2,Spain,80110,0.0,6803.0,0.0,14709.0,58598,4165.0,1713.0,146.0,Jan 30,2020-03-30 05:03:32.157685,-3.647550,40.244487,21512.0,0.316242,0.084921,0.116065,0.009856,Last 8 columns are added by Roaya
5,Iran,38309,0.0,2640.0,0.0,12391.0,23278,3206.0,456.0,31.0,Feb 18,2020-03-30 05:03:32.157685,54.274070,32.575033,15031.0,0.175637,0.068913,0.081098,0.005589,Last 8 columns are added by Roaya
4,France,40174,0.0,2606.0,0.0,7202.0,30366,4632.0,615.0,40.0,Jan 23,2020-03-30 05:03:32.157685,2.618787,47.824905,9808.0,0.265701,0.064868,0.052918,0.003433,Last 8 columns are added by Roaya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,Bahamas,14,0.0,0.0,0.0,1.0,13,0.0,36.0,0.0,Mar 14,2020-03-30 05:03:32.157685,-76.628430,24.290367,1.0,0.000000,0.000000,0.000005,0.000000,Last 8 columns are added by Roaya
136,Tanzania,14,0.0,0.0,0.0,1.0,13,0.0,0.2,0.0,Mar 15,2020-03-30 05:03:32.157685,34.813100,-6.275654,1.0,0.000000,0.000000,0.000005,0.000000,Last 8 columns are added by Roaya
137,Eritrea,12,0.0,0.0,0.0,0.0,12,0.0,3.0,0.0,Mar 20,2020-03-30 05:03:32.157685,38.846170,15.361866,0.0,0.000000,0.000000,0.000000,0.000000,Last 8 columns are added by Roaya
138,Mongolia,12,0.0,0.0,0.0,2.0,10,0.0,4.0,0.0,Mar 09,2020-03-30 05:03:32.157685,103.052998,46.826815,2.0,0.000000,0.000000,0.000011,0.000000,Last 8 columns are added by Roaya


In [27]:
df_used.to_csv('data.csv')
df_used.to_html('data.html')

In [28]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [29]:
corona_json

'[{"Country":"USA","TotalCases":142735,"NewCases":275.0,"TotalDeaths":2489.0,"NewDeaths":5.0,"TotalRecovered":4562.0,"ActiveCases":135684,"CriticalCases":2970.0,"Tot\\u00a0Cases\\/1M pop":431.0,"Deaths\\/1M pop":8.0,"Reported1st case":"Jan 20","Date":1585544612157,"Longitude":-97.922211,"Latitude":39.381266,"ClosedCases":7051.0,"ClosedDeathRate":0.3529995745,"CurrentDeathRate":0.0174379094,"Weight":0.0380427745,"DeathRateXWeight":0.0006633865,"notes":"Last 8 columns are added by Roaya"},{"Country":"Italy","TotalCases":97689,"NewCases":0.0,"TotalDeaths":10779.0,"NewDeaths":0.0,"TotalRecovered":13030.0,"ActiveCases":73880,"CriticalCases":3906.0,"Tot\\u00a0Cases\\/1M pop":1616.0,"Deaths\\/1M pop":178.0,"Reported1st case":"Jan 29","Date":1585544612157,"Longitude":12.07001339,"Latitude":42.79662641,"ClosedCases":23809.0,"ClosedDeathRate":0.45272796,"CurrentDeathRate":0.1103399564,"Weight":0.128458434,"DeathRateXWeight":0.014174098,"notes":"Last 8 columns are added by Roaya"},{"Country":"Spa